In [37]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [38]:
import numpy as np
import tensorflow as tf
from utils_functions import *
from plan import *
import random
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorboard.plugins.hparams import api as hp
import datetime, os

In [39]:
# controllo di star effettivamente usando la GPU

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [40]:
dict_stati= load_file("./dizionario_stati")
plansLoaded=load_file("./plans")

File loaded
File loaded


In [41]:
#ogni stato è un tensore con elementi dtype int8, questi stati vengono raggruppati in un altro tensore che rappresnta la
#variabile "states" del piano 

def build_vector(dict, states_list):
    l=len(dict)
    vector_states=[]
    for state in states_list:
        vector=np.array([0]*l,dtype=np.int8)
        #vector=tf.zeros(l,dtype=np.int8)
        for s in state:
            for key in dict.keys():
                if key==s:
                    vector[dict[key]-1]=1
                    break
        #t=tf.convert_to_tensor(vector,dtype=tf.int8)            
        vector_states.append(vector) 
    r = np.array(vector_states)                          
    return r

#NON UTILIZZATO ADESSO
#shape di ogni singolo elemento (r) è (n x 340) con n che varia su ogni piano, raggruppati sulla base dei piani

def build_all_vectors(dict,plans_list):
    total=[]
    for plan in plans_list:
        r=build_vector(dict,plan.states)
        total.append(r)
    #r = tf.convert_to_tensor(total,dtype=None)                         
    return total
 
    
#gli stati vengono ordinati (in una lista) singolarmente con shape (1x340), non vengono raggruppati sulla base dei piani
#da utilizzare per autoencoder standard

def build_all_vectors1x340(dict, plans_list):
    l=len(dict)
    total=[]
    for plan in plans_list:
        for state in plan.states:
            vector=np.array([0]*l,dtype=np.int8)
            for s in state:
                for key in dict.keys():
                    if key==s:
                        vector[dict[key]-1]=1
                        break
            #t=tf.convert_to_tensor(vector,dtype=tf.int8)            
            total.append(vector)
    total=np.array(total)        
    return total        

## Preparazione dati

### Caricamento dataset statici

In [42]:
test=load_file("./Dataset/set_test")
train=load_file("./Dataset/set_training")
validation=load_file("./Dataset/set_validation")

File loaded
File loaded
File loaded


In [43]:
test=np.array(test,dtype=np.int8)
train=np.array(train,dtype=np.int8)
validation=np.array(validation,dtype=np.int8)

In [44]:
logdir = os.path.join("./TestLogs/", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10),
    tf.keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=1, write_graph=True, write_images=True)
]

In [45]:
# Per il test mi limito a tunare il numero di unità nascoste.
# Se poi funziona, dovrebbe trattarsi solo di scrivere meccanicamente le righe restanti.
input_size = 340
HP_HIDDEN_SIZE = hp.HParam('hidden_size', hp.Discrete([160, 180]))
code_size = 85
METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('./logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_HIDDEN_SIZE],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [46]:
def train_model(hparams):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(input_size,)),
        tf.keras.layers.Dense(hparams[HP_HIDDEN_SIZE], activation=tf.nn.swish),
        tf.keras.layers.Dense(code_size, activation=tf.nn.swish),
        tf.keras.layers.Dense(hparams[HP_HIDDEN_SIZE], activation=tf.nn.swish),
        tf.keras.layers.Dense(input_size, activation=tf.nn.sigmoid),
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy'],
    )
    
    model.fit(
        x=train,y=train,
        epochs=15,
        batch_size=1000,
        validation_data=(validation,validation),
        callbacks=my_callbacks,
    )
    _, accuracy = model.evaluate(x=test, y=test)
    
    return accuracy
    

In [47]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        accuracy = train_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [48]:
session_num = 0

for h_units in HP_HIDDEN_SIZE.domain.values:
    hparams= {
        HP_HIDDEN_SIZE: h_units,
    }
    run_name = "run-%d" % session_num
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})
    run('logs/hparam_tuning/' + run_name, hparams)
    session_num += 1

--- Starting trial: run-0
{'hidden_size': 160}
Epoch 1/15
923/923 [==============================] - 6s 6ms/step - loss: 0.1306 - accuracy: 0.1051 - val_loss: 0.0619 - val_accuracy: 0.1093
Epoch 2/15
923/923 [==============================] - 5s 5ms/step - loss: 0.0454 - accuracy: 0.1185 - val_loss: 0.0357 - val_accuracy: 0.1372
Epoch 3/15
923/923 [==============================] - 5s 5ms/step - loss: 0.0323 - accuracy: 0.1484 - val_loss: 0.0296 - val_accuracy: 0.1500
Epoch 4/15
923/923 [==============================] - 5s 5ms/step - loss: 0.0277 - accuracy: 0.1652 - val_loss: 0.0261 - val_accuracy: 0.1876
Epoch 5/15
923/923 [==============================] - 5s 5ms/step - loss: 0.0248 - accuracy: 0.1913 - val_loss: 0.0236 - val_accuracy: 0.2107
Epoch 6/15
923/923 [==============================] - 5s 5ms/step - loss: 0.0226 - accuracy: 0.2140 - val_loss: 0.0217 - val_accuracy: 0.2283
Epoch 7/15
923/923 [==============================] - 4s 5ms/step - loss: 0.0207 - accuracy: 0.2272 -

In [50]:
%tensorboard --logdir ./logs/hparam_tuning

'kill' is not recognized as an internal or external command,
operable program or batch file.


Reusing TensorBoard on port 6006 (pid 3680), started 2 days, 16:56:40 ago. (Use '!kill 3680' to kill it.)